# SEMELHANÇA ENTRE PRODUTOS

* [1. Carregando Pacotes e Fazendo Especificações](#chapter1)
* [2. Importando os Dados e Fazendo as primeiras análises](#chapter2)
    * [2.1. Convertendo os DataFrames em listas](#section_2_1)
    * [2.2. Particionando os DataFrames](#section_2_2)
    * [2.3. Loop em for para trazer os resultados esperados](#section_2_3)
* [3. Trabalhando com matrizes](#chapter3)
* [4. Usando Computação Distribuída](#chapter4)
    * [4.1. Utilizando PySpark](#section_4_1)
    * [4.2. Utilizando a função Jaro_Winkler para calcular a similaridade](#section_4_2)
    * [4.3. Ordenando os resultados](#section_4_3)

# 1. Carregando Pacotes e Fazendo Especificações <a class="anchor" id="chapter1"></a>

In [1]:
import numpy as np 
import pandas as pd 
import csv
from datetime import datetime

import ray
import jellyfish
import sys
import itertools
import collections.abc

from time import sleep
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

fazer a especificação unicode é essencial para poder fazer as comparações de strings com a função do pacote jellyfish

In [2]:
import sys
if sys.version_info[0] >= 3:
    unicode = str

In [3]:
import os
os.chdir('/Users/brunnokalyxton/Desktop/Data Science/13 - Mercado Livre/Desafio ds 2022')
os.getcwd()

'/Users/brunnokalyxton/Desktop/Data Science/13 - Mercado Livre/Desafio ds 2022'

In [4]:
pd.options.display.max_rows
pd.set_option('display.max_rows', None)

# 2. Importando os Dados e Fazendo as primeiras análises <a class="anchor" id="chapter2"></a>

In [5]:
df= pd.read_csv("items_titles.csv")
df.shape

(30000, 1)

In [6]:
df_teste= pd.read_csv("items_titles_test.csv")
df_teste.shape

(10000, 1)

## 2.1. Convertendo os DataFrames em listas <a class="anchor" id="section_2_1"></a>

In [7]:
df_list = df.values.tolist()
df_teste_list = df.values.tolist()

## 2.2. Particionando os DataFrames <a class="anchor" id="section_2_2"></a>

Irei trabalhar com apenas 1000 linhas em cada dataframe para testar o tempo de processamento da solução que será criada

In [8]:
partialA = df_list[0:1000] #arquivo parcial de items_titles.csv
partialB = df_teste_list[0:1000] #arquivo parcial de items_titles_test.csv

É preciso que cada string esteja em unicode para que seja processada pela função jaro_distance

In [9]:
PartialListA = [unicode(i) for i in partialA] 
PartiallistB = [unicode(i) for i in partialB]

## 2.3. Loop em for para trazer os resultados esperados <a class="anchor" id="section_2_3"></a>

Foi desenvolvido dois loops for para que cada linha de um dataframe interasse com todas as linhas do outro dataframe. Desta maneira, como estamos com dois dataframes de 1000 linhas, é esperado um output de 1.000.000 de linhas. 

Irei acompanhar o tempo de processamento com a função time

In [10]:
%%time
res    = []
lista1 = []
lista2 = []
for nickB in PartiallistB:
    for nickA in PartialListA:
        results = jellyfish.jaro_distance(nickA, nickB)
        res.append(results)
        lista1.append(nickA)
        lista2.append(nickB)
        
#         print (nickB,nickA,results)

CPU times: user 9.9 s, sys: 83.2 ms, total: 9.98 s
Wall time: 10.1 s


O tempo de processamento para as 1000 entradas em cada dataframe foi de 8.33 segundos. Isso gera um problema para quando formos utilizar as 30000 linhas, pois irá gerar um output de 900.000.000. Há possibilidade de então trabalharmos com computação distribuida

In [11]:
final = list(zip(lista1,lista2, res))
simil = pd.DataFrame(final, columns=['A','B','Resultado'])
simil.drop(simil.loc[simil['Resultado']==1].index, inplace=True)
simil = simil.sort_values('Resultado', ascending = False)
simil.head(100)

,A,B,Resultado
209912,['Tênis Via Marte 21-7202 Knit'],['Tênis Via Marte 217202'],0.937500
912209,['Tênis Via Marte 217202'],['Tênis Via Marte 21-7202 Knit'],0.937500
228968,['Tênis Usthemp Volare Temá\x81tico - Praia 2'],['Tênis Usthemp Volare Temá\x81tico - Snes 3'],0.921215
968228,['Tênis Usthemp Volare Temá\x81tico - Snes 3'],['Tênis Usthemp Volare Temá\x81tico - Praia 2'],0.921215
658008,['Tênis Usthemp Short Temá\x81tico - Maria Vir...,['Tênis Usthemp Slip-on Temá\x81tico - Maria V...,0.920868
8658,['Tênis Usthemp Slip-on Temá\x81tico - Maria V...,['Tênis Usthemp Short Temá\x81tico - Maria Vir...,0.920868
385275,['Tênis Usthemp Long Temá\x81tico - Brigadeiro...,['Tênis Usthemp Long Temá\x81tico - Poodle Bra...,0.905229
275385,['Tênis Usthemp Long Temá\x81tico - Poodle Bra...,['Tênis Usthemp Long Temá\x81tico - Brigadeiro...,0.905229
469998,['Lançamento Tênis Caterpillar Intruder Origin...,['Lançamento Tênis Caterpillar Desert Original...,0.898652
998469,['Lançamento Tênis Caterpillar Desert Original...,['Lançamento Tênis Caterpillar Intruder Origin...,0.898652


Após o primeiro processamento, transformo a lista em um dataframe de nome simil. Também são retirados as linhas onde os resultados são iguais a 1, pois representam comparação com resultado identico no outro dataframe. Por fim os dados são ordenamos pelo maior resultado ao menor.

# 3. Trabalhando com matrizes <a class="anchor" id="chapter3"></a>

Para tentar diminuir o tempo de processamento, irei transformar as listas parciais em matrizes e verificar se há melhora no desempenho

In [12]:
matPartialA = np.array(partialA)
matPartialB = np.array(partialB)

In [13]:
%%time
matRes    = []
matLista1 = []
matLista2 = []
for matNickB in matPartialB:
    for matNickA in matPartialA:
        matResults = jellyfish.jaro_distance(str(matNickA), str(matNickB))
        matRes.append(matResults)
        matLista1.append(matNickA)
        matLista2.append(matNickB)

CPU times: user 54.7 s, sys: 278 ms, total: 55 s
Wall time: 55.2 s


In [14]:
matFinal = list(zip(matLista1,matLista2, matRes))
matSimil = pd.DataFrame(matFinal, columns=['A','B','Resultado'])
matSimil.drop(simil.loc[simil['Resultado']==1].index, inplace=True)
matSimil = simil.sort_values('Resultado', ascending = False)
matSimil.head(100)

,A,B,Resultado
209912,['Tênis Via Marte 21-7202 Knit'],['Tênis Via Marte 217202'],0.937500
912209,['Tênis Via Marte 217202'],['Tênis Via Marte 21-7202 Knit'],0.937500
228968,['Tênis Usthemp Volare Temá\x81tico - Praia 2'],['Tênis Usthemp Volare Temá\x81tico - Snes 3'],0.921215
968228,['Tênis Usthemp Volare Temá\x81tico - Snes 3'],['Tênis Usthemp Volare Temá\x81tico - Praia 2'],0.921215
658008,['Tênis Usthemp Short Temá\x81tico - Maria Vir...,['Tênis Usthemp Slip-on Temá\x81tico - Maria V...,0.920868
8658,['Tênis Usthemp Slip-on Temá\x81tico - Maria V...,['Tênis Usthemp Short Temá\x81tico - Maria Vir...,0.920868
385275,['Tênis Usthemp Long Temá\x81tico - Brigadeiro...,['Tênis Usthemp Long Temá\x81tico - Poodle Bra...,0.905229
275385,['Tênis Usthemp Long Temá\x81tico - Poodle Bra...,['Tênis Usthemp Long Temá\x81tico - Brigadeiro...,0.905229
469998,['Lançamento Tênis Caterpillar Intruder Origin...,['Lançamento Tênis Caterpillar Desert Original...,0.898652
998469,['Lançamento Tênis Caterpillar Desert Original...,['Lançamento Tênis Caterpillar Intruder Origin...,0.898652


In [15]:
simil.shape

(999000, 3)

In [16]:
matSimil.shape

(999000, 3)

O tempo de processamento quando trabalhando com matrizes subiu muito. Ainda assim, em tentativas de realizar o procedimento com todos os dados o tempo de processamento foi superior a 10 minutos, motivo pelo qual iremos trabalhar com computação distribuída.

# 4. Usando Computação Distribuída <a class="anchor" id="chapter4"></a>

Para trabalhar com computação distribuída irei transformar a solução do problema em uma função de dois argumentos, onde estes serão os dataframes (de 1 coluna com n linhas) que venham a ser comparados.

A função irá realizar os seguintes procedimentos
   * Iniciar 3 listas vazias (res, lista1, lista2)
   * transformar os dois dataframes que entram na função em lista
   * Aplicar Unicode nas duas listas geradas a partir do dataframe
   * Para cada frase contida em cada lista, aplicar a função jaro_distance para comparar com todas as outras frases contidas na outra lista disponível, gerando assim os valores de resultados.
   * Realizar o append dos resultados nas listas res, lista1 e lista2.
   * Criar um dataframe a partir da concatenação das listas res, lista1 e lista2.
   * Retirar do dataframe as linhas com resultados iguais a 1; e 
   * Ordenar a lista pela coluna Resultados, do maior para o menor.

In [17]:
%%time
def similitud(x,y):
    res    = []
    lista1 = []
    lista2 = []
    xList = x.values.tolist()
    yList = y.values.tolist()
    UniListA = [unicode(i) for i in xList] 
    UniListB = [unicode(i) for i in yList]
    for titleB in UniListB:
        for titleA in UniListA:
            results = jellyfish.jaro_distance(titleA, titleB)
            res.append(results)
            lista1.append(titleA)
            lista2.append(titleB)
    final = list(zip(lista1,lista2, res))
    simil = pd.DataFrame(final, columns=['A','B','Resultado'])
    simil.drop(simil.loc[simil['Resultado']==1].index, inplace=True)
    simil = simil.sort_values('Resultado', ascending = False)
    return simil.head(100)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 9.06 µs


In [18]:
%%time
similitud(df[0:100],df_teste[0:100])

CPU times: user 126 ms, sys: 9.81 ms, total: 135 ms
Wall time: 135 ms


,A,B,Resultado
4308,['Tênis Usthemp Short Temá\x81tico - Maria Vir...,['Tênis Usthemp Short Temá\x81tico - Bulldog F...,0.839214
2259,['Tênis Skechers Go Walk Joy Marinho'],['Tênis Skechers M Go Walk Evolution Ultra'],0.838836
208,['Tênis Usthemp Short Temá\x81tico - Maria Vir...,['Tênis Usthemp Slip-on Temá\x81tico - Labrado...,0.829789
2121,['Tenis Feminino De Fazer Caminhada Corrida Ac...,['Tenis Feminino Academia Caminhada Crossfit C...,0.808324
7935,['Tênis Feminino Top Star Injetado Preto'],['Tênis Feminino Verde Militar Loucos & Santos'],0.794974
851,['Tenis Infantil Preto Molekinho/ Dia Das Cria...,['Tenis Infantil Feminino Menina Criança Moça'],0.791553
6565,['Tênis Feminino Beira Rio Esportivo Sem Cadar...,['Tênis Feminino Em Napa Branco Com Recortes P...,0.790707
7415,['Tênis Infantil Feminino Kidy Hype Nude'],['Tênis Infantil Feminino Luna Cano Médio Now ...,0.789873
7463,['Tênis Infantil Menina Fashion Kids Moda Femi...,['Tênis Infantil Feminino Luna Cano Médio Now ...,0.789704
6438,['Tenis Due Sport '],['Tenis Nike Shadow'],0.788889


A função criada e testada com as 100 primeiras linhas dos dois dataframes está bem sucedida.

## 4.1. Utilizando PySpark <a class="anchor" id="section_4_1"></a>

Iniciando Sessão PySpark

In [21]:
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("similitud") \
    .getOrCreate()

Convertendo os dataframes iniciais em dataframes pyspark

In [22]:
sparkDF =spark.createDataFrame(df) 
sparkDF_test=spark.createDataFrame(df_teste)
sparkDF_test = sparkDF_test.selectExpr("ITE_ITEM_TITLE as TITLE")
sparkDF.printSchema()
sparkDF.show()

root
 |-- ITE_ITEM_TITLE: string (nullable = true)

+--------------------+
|      ITE_ITEM_TITLE|
+--------------------+
|Tênis Ascension P...|
|Tenis Para Caminh...|
|Tênis Feminino Le...|
|Tênis Olympikus E...|
|Inteligente Led B...|
|Tênis Casual Masc...|
|Tênis Infantil Or...|
|Tv Samsung Qled 8...|
|Tênis Usthemp Sho...|
|Sapatênis West Co...|
|Sapatilha Bike Ab...|
|Sapatenis Casual ...|
|Kit Tênis Slip Fe...|
|adidas Superstar ...|
|Tênis Molekinha C...|
|Tênis Infantil Fe...|
|Bicicleta Masculi...|
|Tenis  Feminino P...|
|Smart Tv Led 40 P...|
|Tênis Actvitta Se...|
+--------------------+
only showing top 20 rows



Fazendo o cross join entre os dois conjuntos de dados

In [36]:
similitud = sparkDF.crossJoin(sparkDF_test)
similitud.show()

+--------------------+--------------------+
|      ITE_ITEM_TITLE|               TITLE|
+--------------------+--------------------+
|Tênis Ascension P...|Tênis Olympikus E...|
|Tênis Ascension P...|Bicicleta Barra F...|
|Tênis Ascension P...|Tênis Usthemp Sli...|
|Tênis Ascension P...|Tênis Casual Femi...|
|Tênis Ascension P...|Tênis Star Baby S...|
|Tênis Ascension P...|Tênis Oakley Freq...|
|Tênis Ascension P...|Tênis Jogging Fem...|
|Tênis Ascension P...|Under Armour Hovr...|
|Tênis Ascension P...|Tenis Infantil Fe...|
|Tênis Ascension P...|Tênis Labellamafi...|
|Tênis Ascension P...|Tênis Feminino In...|
|Tênis Ascension P...|Tênis Masculino S...|
|Tênis Ascension P...|Sapatilhas Sapato...|
|Tênis Ascension P...|Tenis Ous Emergen...|
|Tênis Ascension P...|1879202 Tênis Ram...|
|Tênis Ascension P...|Sapatênis Couro L...|
|Tênis Ascension P...|Tenis Feminino Ca...|
|Tênis Ascension P...|Tênis Fila D-form...|
|Tênis Ascension P...|Tênis Infantil Pé...|
|Tênis Ascension P...|Tênis Femi

Verificando a dimensão de cada dataframe em pyspark

In [27]:
print((sparkDF.count(), len(sparkDF.columns)))

(30000, 1)


In [24]:
print((sparkDF_test.count(), len(sparkDF_test.columns)))

(10000, 1)


In [26]:
print((similitud.count(), len(similitud.columns)))

(300000000, 2)


## 4.2. Utilizando a função Jaro_Winkler para calcular a similaridade <a class="anchor" id="section_4_2"></a>

In [28]:
@udf(DoubleType())
def jaro_winkler(s1, s2):
    return jellyfish.jaro_winkler(s1, s2)

In [37]:
similitud = similitud.withColumn('Score Similitud (0,1)',jaro_winkler(col('ITE_ITEM_TITLE'),col('TITLE')))

In [38]:
similitud.show()

+--------------------+--------------------+---------------------+
|      ITE_ITEM_TITLE|               TITLE|Score Similitud (0,1)|
+--------------------+--------------------+---------------------+
|Tênis Ascension P...|Tênis Olympikus E...|   0.8229761904761904|
|Tênis Ascension P...|Bicicleta Barra F...|   0.5815016034710432|
|Tênis Ascension P...|Tênis Usthemp Sli...|   0.6544096133751306|
|Tênis Ascension P...|Tênis Casual Femi...|   0.6547803617571059|
|Tênis Ascension P...|Tênis Star Baby S...|   0.6056529516994633|
|Tênis Ascension P...|Tênis Oakley Freq...|    0.607008547008547|
|Tênis Ascension P...|Tênis Jogging Fem...|   0.6156060606060606|
|Tênis Ascension P...|Under Armour Hovr...|   0.6084608799863037|
|Tênis Ascension P...|Tenis Infantil Fe...|   0.5345264577013819|
|Tênis Ascension P...|Tênis Labellamafi...|   0.6202645502645503|
|Tênis Ascension P...|Tênis Feminino In...|   0.6313794289645683|
|Tênis Ascension P...|Tênis Masculino S...|   0.6725067385444743|
|Tênis Asc

## 4.3. Ordenando os resultados <a class="anchor" id="section_4_3"></a>

In [ ]:
similitud = df1_x_df2.sort(col("Jaro_Winkler").desc())

In [ ]:
similitud.show(30, truncate=False)

Com os comandos acima a lista final é ordenada conforme o score da função de similaridade Jaro Winkler, do maior para o menor valor